# Scraping bantennews.com

## Import Library

**requests** : digunakan untuk membuat permintaan HTTP ke situs web

**BeautifulSoup** : digunakan untuk mengurai konten HTML



In [46]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Scraping

Fungsi ini melakukan iterasi melalui setiap halaman berita dalam rentang yang ditentukan oleh hal.
Pada setiap halaman, fungsi mengambil konten HTML menggunakan requests.get() dari tautan yang dibentuk berdasarkan nomor halaman.
Jika respons dari permintaan adalah 200 (artinya halaman berhasil dimuat), konten halaman di-parse menggunakan BeautifulSoup untuk ekstraksi informasi.
Fungsi mencari blok konten berita dengan class widget berita-list.
Di dalam blok konten tersebut, ia mencari semua elemen berita dengan class berita.
Untuk setiap artikel berita, fungsi mengekstrak judul dari elemen h3 dan tautan dari elemen a di dalamnya.
Selanjutnya, fungsi melakukan permintaan HTTP GET ke tautan artikel untuk mengambil konten dari setiap artikel secara individu.
Jika konten artikel berhasil diambil (status respons 200), fungsi mengekstrak konten dari elemen dengan class entry-content dan mengumpulkannya.
Setelah mengumpulkan judul, konten, dan kategori dari setiap artikel, informasi tersebut dimasukkan ke dalam list data.
Akhirnya, fungsi mengembalikan list data yang berisi informasi yang telah berhasil di-scrape dari halaman-halaman berita.










In [47]:
def scrap_beritajatim(link, hal, kategori):
    category = kategori
    data = []
    base_url = link
    for page in range(1, hal + 1):
        url = f'{base_url}page/{page}/'
        # print(f"Scraping page: {url}")
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            blok = soup.find('div', class_='td-ss-main-content')
            articles = blok.find_all('div', class_='td-block-span6')
            for article in articles:
                link = article.find('a')['href']
                # print(f"link: {link}")
                title = article.find('h3').text.strip()
                date =article.find('time')
                print(title)
            content_response = requests.get(link)
            if content_response.status_code == 200:
                content_soup = BeautifulSoup(content_response.content, "html.parser")
                article_content = content_soup.find('div', class_='td-ss-main-content')
                if article_content:
                    content_text = ""
                    for paragraph in article_content.find_all(['p']):
                        content_text += paragraph.get_text(strip=True).replace('\n', '').replace('ADVERTISEMENT','').replace('SCROLL TO RESUME CONTENT','')
                    # print(content_text)
                    data.append([title,date,content_text,category])
    return data



Scraping dilakukan perkategori
Hasil scraping perkategori kemudian dirubah menjadi dataframe dan digabungkan.
Data hasil penggabungan kemudian dilakukan random agar datanya tidak tersusun urut berdasarkan jenis kategori.
Kemudian data disimpan.






In [48]:
pendidikan = scrap_beritajatim('https://www.bantennews.co.id/category/hukum/',200,'hukum')

Siswa SD Ditemukan Tewas di Belakang Kampus Unpam
Kejari Serang Kabulkan Penangguhan Penahanan Muhyani Penggembala Kambing
Hotman Paris Minta Keluarga Pengangon Kambing di Serang Jadi Tersangka Menghubunginya
Polda Banten Ungkap Sindikat Penyuntik LPG Bersubsidi Beromzet Miliaran Rupiah
Diduga Korupsi, Kades Mekarjaya di Lebak Dilaporkan Warga ke Kejari
Sepanjang 2023, Tiga Kasus Korupsi Perbankan di Banten
Ada Aroma Pungli di Pasar Rangkasbitung
Maling Gasak 9 Kambing Milik Warga Carenang Serang
Kedapatan Memiliki Sabu, Pria di Lebak Ditangkap Polisi
Membela Diri Saat Pergoki Maling, Pengangon Kambing di Serang Jadi Tersangka
Jaksa Minta Hakim Tolak Pledoi Kades Tambakbaya
Sempat Buron, Polisi Tangkap Kawanan Spesialis Bobol Rumah di Serang
Akun Penyebar Video Porno Diduga ASN Pemprov Banten Lebih Dari Satu
Awal Tahun Bakal Ada Tersangka Kasus Kredit Fiktif di BJB Labuan
Polisi Sita Ribuan Obat Tanpa Izin Edar dari Tiga Pria di...
Korupsi Dana BSM, Mantan Kepsek SMA 3 Pandeglang Ditun

KeyboardInterrupt: ignored

In [ ]:
sport = scrap_beritajatim('https://www.bantennews.co.id/category/olahraga/',200,'Sport')

In [ ]:
politik = scrap_beritajatim('https://www.bantennews.co.id/category/politik/',200,'Politik')

In [ ]:
hasil1 = pd.DataFrame(pendidikan, columns=['Judul','date','Isi', 'Kategori'])
hasil2 = pd.DataFrame(politik, columns=['Judul','date','Isi', 'Kategori'])
hasil3 = pd.DataFrame(sport, columns=['Judul','date','Isi', 'Kategori'])

In [ ]:
combined_df = pd.concat([hasil1]).reset_index(drop=True)
combined_df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = combined_df.sample(frac=1).reset_index(drop=True)
df.to_csv('beritabanten.csv', index=False)

In [ ]:
df

In [ ]:
pd.crosstab(index=df['Kategori'], columns='count')